In [ ]:
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm
from shapely.ops import transform
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# FUNCIONES
# -------------------------
def drop_z(geom):
    if geom is None:
        return None
    return transform(lambda x, y, z=None: (x, y), geom)

# -------------------------
# CARGA DE DATOS
# -------------------------
atlantico = gpd.read_file("MAPAS/MGN_2/DPTO_CNMBR_ATLÁNTICO.shp")
atlantico = atlantico.rename(columns={
    "MPIO_CCNCT": "COD_MUN",
    "MPIO_CNMBR": "NOM_MUN"
})
atlantico = atlantico[["COD_MUN", "NOM_MUN", "MPM", "geometry"]]

vias   = gpd.read_file("MAPAS/VIAS/Linea.shp")
puntos = gpd.read_file("MAPAS/PUNTOS/Puntos.shp")
nuevas = gpd.read_file("MAPAS/NUEVAS/Vías Nuevas.shp")
